In [ ]:
!unzip Segmentation_dataset.zip

Archive:  Segmentation_dataset.zip
   creating: Segmentation_dataset/
  inflating: __MACOSX/._Segmentation_dataset  
  inflating: Segmentation_dataset/.DS_Store  
  inflating: __MACOSX/Segmentation_dataset/._.DS_Store  
   creating: Segmentation_dataset/After_annotating/
  inflating: __MACOSX/Segmentation_dataset/._After_annotating  
  inflating: Segmentation_dataset/After_annotating/.DS_Store  
  inflating: __MACOSX/Segmentation_dataset/After_annotating/._.DS_Store  
   creating: Segmentation_dataset/After_annotating/image/
  inflating: __MACOSX/Segmentation_dataset/After_annotating/._image  
   creating: Segmentation_dataset/After_annotating/mask/
  inflating: __MACOSX/Segmentation_dataset/After_annotating/._mask  
  inflating: Segmentation_dataset/After_annotating/image/32I2.png  
  inflating: __MACOSX/Segmentation_dataset/After_annotating/image/._32I2.png  
  inflating: Segmentation_dataset/After_annotating/image/81I.png  
  inflating: __MACOSX/Segmentation_dataset/After_annotating

In [ ]:
#Change Segmentation_dataset/image to images and mask to masks
!ls
!rm -r Augmented_segmentation
!mkdir Augmented_segmentation
!mkdir Augmented_segmentation/train
!mkdir Augmented_segmentation/test
!mkdir Augmented_segmentation/train/images
!mkdir Augmented_segmentation/train/masks
!mkdir Augmented_segmentation/test/images
!mkdir Augmented_segmentation/test/masks

Augmented_segmentation	sample_data	      Segmentation_dataset.zip
__MACOSX		Segmentation_dataset


In [ ]:
import os
import cv2
from tqdm import tqdm
from glob import glob
from albumentations import RandomRotate90, ColorJitter, VerticalFlip, HorizontalFlip

def load_data(path):
    images = sorted(glob(os.path.join(path, "images/*")))
    masks = sorted(glob(os.path.join(path, "masks/*")))
    return images, masks

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def augment_data(images, masks, save_path, augment=True):
    H = 256
    W = 256
    p = 0
    for x, y in tqdm(zip(images, masks), total=len(images)):
        name = x.split("/")[-1].split(".")
        """ Extracting the name and extension of the image and the mask. """
        image_name = name[0]
        image_extn = name[1]

        name = y.split("/")[-1].split(".")
        mask_name = name[0]
        mask_extn = name[1]

        """ Reading image and mask. """
        x = cv2.imread(x, cv2.IMREAD_COLOR)
        y = cv2.imread(y, cv2.IMREAD_COLOR)

        """ Augmentation """
        if augment == True:
            # aug = CenterCrop(H, W, p=1.0)
            # augmented = aug(image=x, mask=y)
            # x1 = augmented["image"]
            # y1 = augmented["mask"]

            aug = ColorJitter(brightness=(0.5,0.75), contrast=(0.5,0.75), p=1.0)
            augmented = aug(image=x, mask=y)
            x1 = augmented['image']
            y1 = augmented['mask']

            aug = RandomRotate90(p=1.0)
            augmented = aug(image=x, mask=y)
            x2 = augmented['image']
            y2 = augmented['mask']

            aug = HorizontalFlip(p=1.0)
            augmented = aug(image=x2, mask=y2)
            x3 = augmented['image']
            y3 = augmented['mask']

            # aug = VerticalFlip(p=1.0)
            # augmented = aug(image=x2, mask=y2)
            # x3 = augmented['image']
            # y3 = augmented['mask']

            save_images = [x, x1, x3]
            save_masks =  [y, y1, y3]

        else:
            save_images = [x]
            save_masks = [y]

        """ Saving the image and mask. """
        idx = 0
        for i, m in zip(save_images, save_masks):
            # i = cv2.resize(i, (W, H))
            # m = cv2.resize(m, (W, H))

            if len(images) == 1:
                tmp_img_name = f"{image_name}.{image_extn}"
                tmp_mask_name = f"{mask_name}.{mask_extn}"
            else:
                tmp_img_name = f"{image_name}_{idx}.{image_extn}"
                tmp_mask_name = f"{mask_name}_{idx}.{mask_extn}"

            if p<=800:
              image_path = os.path.join(save_path,"train", "images", tmp_img_name)
              mask_path = os.path.join(save_path,"train", "masks", tmp_mask_name)

              cv2.imwrite(image_path, i)
              cv2.imwrite(mask_path, m)

              idx += 1
              p+=1
            else:
              image_path = os.path.join(save_path,"test", "images", tmp_img_name)
              mask_path = os.path.join(save_path,"test", "masks", tmp_mask_name)

              cv2.imwrite(image_path, i)
              cv2.imwrite(mask_path, m)

              idx += 1
              p+=1


In [ ]:
#Load images into a list
path = "/content/Segmentation_dataset/After_annotating"
images, masks = load_data(path)
print(f"Original Images: {len(images)} - Original Masks: {len(masks)}")


# New directory paths
new_path = "/content/Augmented_segmentation"

""" Applying data augmentation. """
augment_data(images, masks, "/content/Augmented_segmentation", augment=True)

""" Loading augmented images and masks. """
images, masks = load_data("/content/Augmented_segmentation/train/")
print(f"Augmented Images: {len(images)} - Augmented Masks: {len(masks)}")

Augmented Images: 225 - Augmented Masks: 225


In [ ]:
from google.colab.patches import cv2_imshow
for imagepaths in os.listdir('/content/Augmented_segmentation/images'):
    imagepaths = '/content/Augmented_segmentation/images/' + imagepaths
    image_cv = cv2.imread(imagepaths)
    i1 = image_cv.shape[0]
    i2 = image_cv.shape[1]
    i3 = image_cv.shape[2]
    #print(i1, i2, i3)
    if i1!=256 or i2!=256 or i3!=3:
      print(image_cv.shape)
    #cv2_imshow(image_cv)

In [ ]:
!zip -r /content/split_datset.zip /content/Augmented_segmentation

  adding: content/Augmented_segmentation/ (stored 0%)
  adding: content/Augmented_segmentation/train/ (stored 0%)
  adding: content/Augmented_segmentation/train/masks/ (stored 0%)
  adding: content/Augmented_segmentation/train/masks/47g_2.png (deflated 86%)
  adding: content/Augmented_segmentation/train/masks/21a_2.png (deflated 78%)
  adding: content/Augmented_segmentation/train/masks/23I_0.png (deflated 78%)
  adding: content/Augmented_segmentation/train/masks/37e_2.png (deflated 82%)
  adding: content/Augmented_segmentation/train/masks/57f_2.png (deflated 85%)
  adding: content/Augmented_segmentation/train/masks/25I_0.png (deflated 85%)
  adding: content/Augmented_segmentation/train/masks/29k_2.png (deflated 85%)
  adding: content/Augmented_segmentation/train/masks/35I_0.png (deflated 83%)
  adding: content/Augmented_segmentation/train/masks/15k2_0.png (deflated 84%)
  adding: content/Augmented_segmentation/train/masks/11k_2.png (deflated 85%)
  adding: content/Augmented_segmentatio